In [153]:
import pandas as pd

file_path = 'mse_434_paper_data.xlsx'

try:
    df = pd.read_excel(file_path, sheet_name='customers_nonstress')
    print("Data loaded successfully:")
    display(df)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

Data loaded successfully:


,Plant,Longitude,Latitude,Daily consumption [T],Safety stock [T]
0,Central,17.00,52.00,—,—
1,2,17.00,54.44,6,6
2,3,15.06,53.50,6,6
3,4,20.52,50.34,12,12
4,5,21.77,50.24,19,19
5,8,22.22,50.92,25,25
6,10,22.82,52.30,20,20
7,12,15.49,53.36,17,17
8,13,22.38,51.93,12,12
9,20,20.01,53.27,7,7


In [154]:
'''
Calculate distance using the Haversine Formula for spherical data / longitude and latitude coordinates
'''

# https://community.esri.com/t5/coordinate-reference-systems-blog/distance-on-a-sphere-the-haversine-formula/ba-p/902128

def haversine(coord1: object, coord2: object):
    import math

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    km = round(km, 3)
    
    return km


In [155]:
n = len(df)
# Convert Plant column to string to ensure consistent types
plant_names = df['Plant'].astype(str).tolist()
distance_matrix = pd.DataFrame(index=plant_names, columns=plant_names)
distance_matrix.index.name = None

# Create distance matrix using haversine() function to show distances between plants and customers (regional warehouses)
for i in range(n):
    for j in range(n):
        coord1 = (df.loc[i, 'Longitude'], df.loc[i, 'Latitude'])
        coord2 = (df.loc[j, 'Longitude'], df.loc[j, 'Latitude'])

        distance_matrix.iloc[i, j] = haversine(coord1, coord2)

# Convert all values to float
distance_matrix = distance_matrix.astype(float)

# Display result
print("Distance Matrix (in kilometers):")
display(distance_matrix)

Distance Matrix (in kilometers):


,Central,2,3,4,5,8,10,12,13,20,28
Central,0.000,271.316,211.806,307.026,386.073,380.964,398.382,182.285,368.592,247.357,423.874
2,271.316,0.000,164.384,514.532,568.155,526.059,453.343,155.584,454.106,236.396,602.020
3,211.806,164.384,0.000,513.228,585.791,565.574,537.020,32.464,522.776,329.214,622.970
4,307.026,514.532,513.228,0.000,89.496,136.146,270.240,481.588,219.303,327.680,125.345
5,386.073,568.155,585.791,89.496,0.000,82.017,240.420,553.586,192.687,358.010,37.833
8,380.964,526.059,565.574,136.146,82.017,0.000,158.942,533.115,112.853,301.751,94.518
10,398.382,453.343,537.020,270.240,240.420,158.942,0.000,506.103,50.944,217.568,251.349
12,182.285,155.584,32.464,481.588,553.586,533.115,506.103,0.000,491.058,300.378,590.718
13,368.592,454.106,522.776,219.303,192.687,112.853,50.944,491.058,0.000,218.660,207.089
20,247.357,236.396,329.214,327.680,358.010,301.751,217.568,300.378,218.660,0.000,387.313


In [156]:
# === Constants ===
T = 5  # number of time periods
L = 25  # vehicle capacity (tons)
M = 10000  # Big M, large enough to exceed any demand sum
b = 1.2  # inventory holding cost per ton
c_per_km = 2  # transportation cost per km

# === Load customers ===
customers_df = pd.read_excel("mse_434_paper_data.xlsx", sheet_name="customers_nonstress")
customers = [str(i) for i in customers_df["Plant"] if str(i).lower() != "central"]

# === Load valid routes ===
routes_df = pd.read_excel("mse_434_paper_data.xlsx", sheet_name="valid_routes")
route_ids = routes_df["Route ID"].tolist()
route_map = {r: list(map(str, eval(custs))) for r, custs in zip(routes_df["Route ID"], routes_df["Route Combination"])}

# Cost Maps
cj = {}

for r in route_ids:
    route_customers = route_map[r]  
    
    # Calculate actual route distance: Central -> customer1 -> customer2 -> ... -> Central
    total_km = 0
    
    if len(route_customers) > 0:
        # Filter out customers not in distance matrix
        valid_customers = [str(i) for i in route_customers if str(i) in distance_matrix.columns]
        
        if valid_customers:
            # Distance from Central to first customer
            total_km += distance_matrix.loc['Central', valid_customers[0]]
            
            # Distance between consecutive customers
            for i in range(len(valid_customers) - 1):
                current_customer = valid_customers[i]
                next_customer = valid_customers[i + 1]
                total_km += distance_matrix.loc[current_customer, next_customer]
            
            # Distance from last customer back to Central
            total_km += distance_matrix.loc[valid_customers[-1], 'Central']
    
    cj[r] = round(c_per_km * total_km, 2) 
    

# Build aij: parameter for whether customer i is on route j
aij = {(i, j): int(i in route_map[j]) for i in customers for j in route_ids}

print(aij)

# # Build di (demand), gi (safety stock), bi (inventory cost per unit)
di = {i: customers_df.loc[customers_df["Plant"] == int(i), "Daily consumption [T]"].values[0] for i in customers}
gi = {i: customers_df.loc[customers_df["Plant"] == int(i), "Safety stock [T]"].values[0] for i in customers}
bi = {i: b for i in customers}


{('2', 'R1'): 1, ('2', 'R2'): 0, ('2', 'R3'): 0, ('2', 'R4'): 0, ('2', 'R5'): 0, ('2', 'R6'): 0, ('2', 'R7'): 0, ('2', 'R8'): 0, ('2', 'R9'): 0, ('2', 'R10'): 0, ('2', 'R11'): 1, ('2', 'R12'): 1, ('2', 'R13'): 1, ('2', 'R14'): 1, ('2', 'R15'): 1, ('2', 'R16'): 1, ('2', 'R17'): 1, ('2', 'R18'): 0, ('2', 'R19'): 0, ('2', 'R20'): 0, ('2', 'R21'): 0, ('2', 'R22'): 0, ('2', 'R23'): 0, ('2', 'R24'): 0, ('2', 'R25'): 0, ('2', 'R26'): 0, ('2', 'R27'): 0, ('2', 'R28'): 0, ('2', 'R29'): 0, ('2', 'R30'): 0, ('2', 'R31'): 0, ('2', 'R32'): 1, ('2', 'R33'): 1, ('2', 'R34'): 1, ('2', 'R35'): 1, ('2', 'R36'): 1, ('2', 'R37'): 1, ('2', 'R38'): 1, ('2', 'R39'): 1, ('2', 'R40'): 1, ('2', 'R41'): 0, ('2', 'R42'): 0, ('2', 'R43'): 0, ('2', 'R44'): 0, ('2', 'R45'): 0, ('3', 'R1'): 0, ('3', 'R2'): 1, ('3', 'R3'): 0, ('3', 'R4'): 0, ('3', 'R5'): 0, ('3', 'R6'): 0, ('3', 'R7'): 0, ('3', 'R8'): 0, ('3', 'R9'): 0, ('3', 'R10'): 0, ('3', 'R11'): 1, ('3', 'R12'): 0, ('3', 'R13'): 0, ('3', 'R14'): 0, ('3', 'R15'): 

In [157]:
from gurobipy import Model, GRB

model = Model("JointInventoryTransportation")

# === Indices ===
K = range(1, T + 1)

# === Variables ===

# Route usage decision 
x = model.addVars(route_ids, K, vtype=GRB.BINARY, name="x")                  # x_jk

# Inventory level at customer i in time period k
y = model.addVars(customers, K, lb=0, name="y")                              # y_ik

# Delivered amount at customer i in time period k on route j
z = model.addVars(customers, route_ids, K, lb=0, name="z")                   # z_ijk

# Objective: transport cost + inventory cost
model.setObjective(
    sum(cj[j] * x[j, k] for j in route_ids for k in K) +
    sum(bi[i] * y[i, k] for i in customers for k in K),
    GRB.MINIMIZE
)

In [158]:
# === Constraint (1): z_ijk ≤ aij * x_jk * M === (Deliver only if the route is used)
for i in customers:
    for j in route_ids:
        for k in K:
            model.addConstr(z[i, j, k] <= aij[i, j] * x[j, k] * M, name=f"link_z_x_{i}_{j}_{k}")

# === Constraint (2): sum_i z_ijk ≤ Q for each route j, period k === (Truck capacity constraint)
for j in route_ids:
    for k in K:
        model.addConstr(sum(z[i, j, k] for i in customers) <= L, name=f"truck_cap_{j}_{k}")

# === Constraint (3): Inventory balance for period 1 to T-1 === (Inventory continuity constraint for periods 1 to T-1)
for i in customers:
    for k in range(1, T):
        model.addConstr(
            y[i, k+1] == y[i, k] + sum(z[i, j, k] for j in route_ids) - di[i],
            name=f"inventory_flow_{i}_{k}"
        )

# === Constraint (4): Cyclic inventory from T to 1 === (Cyclic inventory constraint for period T to 1)
for i in customers:
    model.addConstr(
        y[i, 1] == y[i, T] + sum(z[i, j, T] for j in route_ids) - di[i],
        name=f"cyclic_inventory_{i}"
    )

# === Constraint (5): Minimum inventory ≥ gi === (Safety stock constraint)
for i in customers:
    for k in K:
        model.addConstr(y[i, k] >= gi[i], name=f"safety_stock_{i}_{k}")
        

In [159]:
model.optimize()

# Output
print(f"\nTotal cost: {model.ObjVal:.2f}\n")

print("Route usage:")
for k in K:
    print(f"\nPeriod {k}:")
    for j in route_ids:
        if x[j, k].X > 0.5:
            print(f"  Route {j} used")

print("\nInventory levels:")
for i in customers:
    for k in K:
        print(f"Customer {i}, Period {k} → {y[i, k].X:.2f} units")

print("\nDeliveries z_ijk:")
for i in customers:
    for j in route_ids:
        for k in K:
            if z[i, j, k].X > 0.01:
                print(f"Deliver {z[i, j, k].X:.2f} to Customer {i} via Route {j} in Period {k}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2575 rows, 2525 columns and 7370 nonzeros
Model fingerprint: 0x6ba46cb7
Variable types: 2300 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 2e+01]
Presolve removed 2320 rows and 1810 columns
Presolve time: 0.01s
Presolved: 255 rows, 715 columns, 1215 nonzeros
Variable types: 500 continuous, 215 integer (215 binary)
Found heuristic solution: objective 443805.12000
Found heuristic solution: objective 441100.71000
Found heuristic solution: objective 438396.30000

Root relaxation: objective 3.575060e+04, 448 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj